In [15]:
import json
import numpy as np
from datetime import date
from xsigmamodules.market.swap_convention_parser import SwapConventionParser
from xsigmamodules.market.xccy_convention_parser import XccyConventionParser
from xsigmamodules.market.future_convention_parser import FutureConventionParser
from xsigmamodules.Market import currencyMappingConfig,currencyMappingConfigBuilder
from xsigmamodules.util.misc import xsigmaGetDataRoot, xsigmaGetTempDir
from xsigmamodules.common import helper #CreateCurrencyMappingConfig
from xsigmamodules.TestingUtil import TestingDataSerializer
from xsigma.util.numpy_support import xsigmaToNumpy, numpyToXsigma
from xsigmamodules.Util import (
    yearMonthDay,
    dayCountConvention,
    calendar,
    day_count_convention_enum,
    weekend_type_enum,
)
from xsigmamodules.Market import currencyMappingConfig,swapDefaultConventionConfig,xccyDefaultConventionConfig,futureDefaultConventionConfig

In [16]:
XSIGMA_DATA_ROOT = xsigmaGetDataRoot() + "/Data/default/"
JSON_DATA_ROOT_CONFIG=xsigmaGetDataRoot() + "/Data/config_data/"
JSON_DATA_ROOT=xsigmaGetDataRoot() + "/Data/"

# STATIC DATA

In [17]:
currency_calendar_mapping = helper.CreateCurrencyMappingConfig(XSIGMA_DATA_ROOT + "currency_calendar_mapping.json",value_filter=".DISCARD",output_file=JSON_DATA_ROOT_CONFIG+"currencyCalendarMapping.json")
currency_rfr_mapping      = helper.CreateCurrencyMappingConfig(XSIGMA_DATA_ROOT + "currency_rfr_mapping.json",value_filter=".DISCARD",output_file=JSON_DATA_ROOT_CONFIG+"currencyRfrMapping.json")
currency_ibor_mapping     = helper.CreateCurrencyMappingConfig(XSIGMA_DATA_ROOT + "currency_ibor_mapping.json",value_filter=".DISCARD",output_file=JSON_DATA_ROOT_CONFIG+"currencyIborMapping.json")

In [18]:
for ccy in currency_calendar_mapping.keys():
    cal_name = currency_calendar_mapping.value(ccy)
    cal = TestingDataSerializer.createCalendar(cal_name, 1900, 2090)
    calendar.write_to_json(JSON_DATA_ROOT_CONFIG + "calendars/calendar_" + cal_name + ".json", cal)

In [19]:
ListOfConfig=SwapConventionParser(XSIGMA_DATA_ROOT + "swap_default_conventions.json")
for ccy, types in ListOfConfig.get_available_currencies().items():
    for t in types:
        config=ListOfConfig.create_swap_convention(ccy, t)
        swapDefaultConventionConfig.write_to_json(JSON_DATA_ROOT_CONFIG + "swapDefaultConventionConfig_" + ccy + "_" + t + ".json", config)

In [20]:
ListOfConfig=XccyConventionParser(XSIGMA_DATA_ROOT + "cross_currency_basis_leg.json")
for ccy in ListOfConfig.get_available_currencies():
    config=ListOfConfig.create_xccy_convention(ccy)
    xccyDefaultConventionConfig.write_to_json(JSON_DATA_ROOT_CONFIG + "xccyDefaultConventionConfig_" + ccy + ".json", config)

In [21]:
ListOfConfig=FutureConventionParser(XSIGMA_DATA_ROOT + "futures_default_conventions.json")
for idx, types in ListOfConfig.get_available_conventions().items():
    for t in types:
        config=ListOfConfig.create_future_convention(idx, t)
        futureDefaultConventionConfig.write_to_json(JSON_DATA_ROOT_CONFIG + "futureDefaultConventionConfig_" + idx + "_" + t.lower() + ".json", config)

# MAKET DATA

In [22]:
valuation_date=yearMonthDay(str(date.today())).to_datetime() #move o a datehelper

In [23]:
ids = [
    # Interest Rate (IR) entries
    ["IR", "USD", "LIBOR.3M.USD"],
    ["IR", "EUR", "LIBOR.3M.USD"],
    ["IR", "GBP", "LIBOR.3M.USD"],
    
    # Foreign Exchange (FX) entries
    ["FX", "USD", "EUR", "LIBOR.3M.USD", "LIBOR.3M.USD"],
    ["FX", "USD", "GBP", "LIBOR.3M.USD", "LIBOR.3M.USD"],
    
    # Credit entries
    ["CREDIT", "APPLE", "EUR", "LIBOR.3M.USD", "a", "b", "c"],
    ["CREDIT", "APPLE", "USD", "LIBOR.3M.USD", "a", "b", "c"],
    
    # Equity entries
    ["EQUITY", "GOOGLE", "USD", "LIBOR.3M.USD"],
    ["EQUITY", "APPLE", "USD", "LIBOR.3M.USD"],
]
factors=[3, 2, 1, 1, 2, 1, 2, 2, 1]
with open(XSIGMA_DATA_ROOT +"correlation_matrix.json") as f:
    matrix = json.load(f)

helper.CreatecorrelationManager(valuation_date, ids, factors, matrix, JSON_DATA_ROOT + "correlationManager.json")